# SQL Project: Answering Business questions about a digital music store

The [Chinook database](https://github.com/lerocha/chinook-database) contains information about a fictional digital music shop - kind of like a mini-iTunes store. The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. 

<img src='mp4.jpg' alt='mp4.jpg' width='300' height='450'>
    
In this project we will answer three questions. 

- Select new albuns in a list to add to the platafform. 
- Analyzing employee's sale performance. 
- Analyzing sales by country.    
    
This information is contained in eleven tables. Here's a schema diagram for the Chinook database:

<img src='chinook-schema.svg' alt='chinook-schema.svg' width='900' height='1200'>

Load the data

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Get familiar with the data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type = "table";

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


---
#### Question 1 

The record store Chinook has just signed a new record label. The new record label specializes in USA artists in a wide variety of genres. They have given Chinook some money to add 3 new albums in a list of four in the USA. Below the names and genres of the 4 artists to be chosen.

| Artist Name | Genre |
|---|---|
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

Our task is select the three albums that will be added to the store. 

---

Our initial approach is create a query that returns each genre and the absolute sale numbers and relative sale numbers in the USA. First we create a view which returns the tracks sold in USA.

In [3]:
%%sql
CREATE VIEW tracks_sold_usa AS
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name customer_name,
        c.country,
        t.track_id,
        t.name song,
        g.name genre
    FROM invoice_line i_l
    INNER JOIN invoice i ON i.invoice_id = i_l.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    INNER JOIN track t ON t.track_id = i_l.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE c.country = 'USA';

    
SELECT *
FROM tracks_sold_usa
LIMIT 5;

 * sqlite:///chinook.db
(sqlite3.OperationalError) table tracks_sold_usa already exists
[SQL: CREATE VIEW tracks_sold_usa AS
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name customer_name,
        c.country,
        t.track_id,
        t.name song,
        g.name genre
    FROM invoice_line i_l
    INNER JOIN invoice i ON i.invoice_id = i_l.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    INNER JOIN track t ON t.track_id = i_l.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE c.country = 'USA';]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


Let's group by the genre and count the number of sold track for genre.

In [4]:
%%sql
SELECT 
    genre, 
    COUNT(genre) count
FROM tracks_sold_usa
GROUP BY genre
ORDER BY count DESC;

 * sqlite:///chinook.db
Done.


genre,count
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Pop,22
Latin,22
Hip Hop/Rap,20
Jazz,14


Selecting only the four genres: Hip-Hop, Punk, Pop and Blues.

In [5]:
%%sql
SELECT 
    genre, 
    COUNT(genre) tracks_sold,
    ROUND(CAST(COUNT(genre) AS FLOAT) / CAST((SELECT COUNT(*) FROM tracks_sold_usa) AS FLOAT),3) tracks_sold_percent
FROM tracks_sold_usa
WHERE genre IN ('Hip Hop/Rap', 'Alternative & Punk', 'Pop', 'Blues')
GROUP BY genre
ORDER BY tracks_sold DESC;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,tracks_sold_percent
Alternative & Punk,130,0.124
Blues,36,0.034
Pop,22,0.021
Hip Hop/Rap,20,0.019


Looking at the best selling music genres in the USA, we should select the albums of the following artists:

| Artist Name | Genre |
|---|---|
| Red Tone | Punk |
| Slim Jim Bites | Blues |
| Meteor and the Girls | Pop |

In [6]:
%%sql
DROP VIEW IF EXISTS tracks_sold_usa;

 * sqlite:///chinook.db
Done.


[]

---
Question 2. Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Which salesman had the best performance?

---

We have to calculate amount of sales (in USD) in the "invoice" table for each employee (employee_id) in the "employee" table. We also also add other information to the next query.

In [26]:
%%sql
SELECT
    e.employee_id,
    e.first_name || ' ' || e.last_name customer_name,
    e.hire_date,
    ROUND(SUM(i.total),2) amount_sales_usd,
    ROUND(CAST(SUM(i.total) AS FLOAT) / CAST((SELECT SUM(total) FROM invoice) AS FLOAT), 2) amount_sales_usd_percent
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY employee_id
ORDER BY amount_sales_usd DESC;

 * sqlite:///chinook.db
Done.


employee_id,customer_name,hire_date,amount_sales_usd,amount_sales_usd_percent
3,Jane Peacock,2017-04-01 00:00:00,1731.51,0.37
4,Margaret Park,2017-05-03 00:00:00,1584.0,0.34
5,Steve Johnson,2017-10-17 00:00:00,1393.92,0.3


Jane Peacock was the sales agent whose customers spent more at the store. However, she was the agent who started the company earlier.

---
Question 3. Which country has a higher average purchase on the site? We assume the country to be the one described in the "customer" table. Countries with only one customer should be grouped by "Others" in the "country" column and will not be considered in the analysis. 

---

To place "Others" as a country always at the botton of the table, we will assign an index 0 for countries with more than one customer and 1 for "Others".

In [134]:
%%sql
DROP VIEW IF EXISTS country_more_1_customer;

CREATE VIEW country_more_1_customer AS
    SELECT
        c.country,
        COUNT(DISTINCT(i.customer_id)) customers,
        ROUND(SUM(i.total),2) total,
        ROUND(AVG(i.total),2) average_order,
        0 country_index
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY country
    HAVING customers > 1;

SELECT * 
FROM country_more_1_customer
ORDER BY total DESC;

 * sqlite:///chinook.db
Done.
Done.
Done.


country,customers,total,average_order,country_index
USA,13,1040.49,7.94,0
Canada,8,535.59,7.05,0
Brazil,5,427.68,7.01,0
France,5,389.07,7.78,0
Germany,4,334.62,8.16,0
Czech Republic,2,273.24,9.11,0
United Kingdom,3,245.52,8.77,0
Portugal,2,185.13,6.38,0
India,2,183.15,8.72,0


Countries with only one customer will be grouped as "Others".

In [135]:
%%sql
DROP VIEW IF EXISTS country_with_1_customer_aux;
DROP VIEW IF EXISTS country_with_1_customer;

CREATE VIEW country_with_1_customer_aux AS
    SELECT
        c.country,
        COUNT(DISTINCT(i.customer_id)) customers,
        ROUND(SUM(i.total),2) total,
        ROUND(AVG(i.total),2) average_order,
        1 country_index
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY country
    HAVING customers == 1;

CREATE VIEW country_with_1_customer AS
    SELECT 
        "Others" country,
        COUNT(customers) customers,
        SUM(total) total,
        ROUND(AVG(average_order),2) average_order,
        1 country_index
    FROM country_with_1_customer_aux
    GROUP BY country_index;

SELECT * FROM country_with_1_customer;

 * sqlite:///chinook.db
Done.
Done.
Done.
Done.
Done.


country,customers,total,average_order,country_index
Others,15,1094.94,7.44,1


In [142]:
%%sql
SELECT 
    country,
    customers,
    total,
    average_order,
    country_index
FROM country_more_1_customer

UNION

SELECT 
    country,
    customers,
    total,
    average_order,
    country_index
FROM country_with_1_customer
ORDER BY country_index, average_order DESC;

 * sqlite:///chinook.db
Done.


country,customers,total,average_order,country_index
Czech Republic,2,273.24,9.11,0
United Kingdom,3,245.52,8.77,0
India,2,183.15,8.72,0
Germany,4,334.62,8.16,0
USA,13,1040.49,7.94,0
France,5,389.07,7.78,0
Canada,8,535.59,7.05,0
Brazil,5,427.68,7.01,0
Portugal,2,185.13,6.38,0
Others,15,1094.94,7.44,1


The Czech Republic has the highest average of orders at chinook store followed by United Kingdom and India.